# Introduction

This notebook implements a gradient-boosted decision tree ensemble for predicting bad loans through XGBoost. This notebook is incomplete and I am still working through it. 

Paper on XGBoost for Default Prediction: https://www.terry.uga.edu/sites/default/files/inline-files/Albanesi_Domossy_2021.pdf

In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import xgboost
import tqdm

from xgboost import XGBClassifier, DMatrix
import multiprocessing

from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [2]:
# Import data and data dictionary
data = pd.read_pickle('output_data/02_data.pkl')

data_dict = pd.read_pickle('output_data/02_data_dict.pkl')

## Train, Test, and Validation Data

TODO: Fix. I'm going to start with a train-validation-test split of 60/20/20. This is weighted more towards test data and less towards training data than others splits that are commonly used. This decision is motivated by concern of overfitting with a powerful model predicting such a small sample size. With a sizeable test set, we should have a clear understanding of how well the model performs on the sample. 

In [48]:
# Split into training, validation, and testing data
# Split chronologically to resolve the look-ahead problem

# Generate a new column to track the observation's model dataset category
data['model_set'] = 'train'

# Validation Data comes from February 2011
data.loc[data['application_month_of_cycle'] == 5, 'model_set'] = 'validation'

# Test Data comes from March 2011
data.loc[data['application_month_of_cycle'] == 6, 'model_set'] = 'test'

# Display results
print("Chronological method achieves Train-Val-Test split of approximately 65-25-10 proportions.")
data['model_set'].value_counts()

Chronological method achieves Train-Val-Test split of approximately 65-25-10 proportions.


train         406
validation    149
test           95
Name: model_set, dtype: int64

TODO: PowerPoint chart to explain where we divided the train, test, and validation data. 

In [58]:
# Split into x and y data
# Use the list of potential features from the data dictionary
potential_features = data_dict.loc[data_dict['potential_feature']==True, 'variable'].values

# Designate X data for train, validation, and test
X_train = data.loc[data['model_set'] == 'train', potential_features]
X_val = data.loc[data['model_set'] == 'validation', potential_features]
X_test = data.loc[data['model_set'] == 'test', potential_features]

# Designate Y data for train, validation, and test
y_train = data.loc[data['model_set'] == 'train', 'bad']
y_val = data.loc[data['model_set'] == 'validation', 'bad']
y_test = data.loc[data['model_set'] == 'test', 'bad']

In [68]:
X_train

,monthly_rent_amount,monthly_income_amount,raw_l2c_score,raw_FICO_telecom,raw_FICO_retail,raw_FICO_bank_card,raw_FICO_money,application_day_of_year,application_month_of_year,application_hour_of_day,rent_to_income_ratio,email_duration_months,residence_duration_months,bank_account_duration_months,payment_frequency_per_month,home_phone_type_home,how_use_money_bills_general,other_phone_type_mobile,how_use_money_bills_medical,expense_type_general,how_use_money_car,home_phone_type_mobile,other_phone_type_home,other_phone_type_work,bank_id_0154,...,bank_id_7955,how_use_money_other,residence_duration_3+_years,bank_id_7751,residence_duration_6_months_or_less,how_use_money_pay_off_loans_overdrawn_acct,bank_id_0215,email_duration_3_months_or_less,payment_frequency_weekly,bank_account_duration_3+_years,how_use_money_won't_say,expense_type_transportation,how_use_money_medical,how_use_money_school,bank_id_0297,bank_id_0005,email_duration_1_year_or_more,bank_account_duration_1_2_years,bank_id_9999,expense_type_credit,residence_duration_7_12_months,expense_type_medical,expense_type_education,bank_account_duration_6_months_or_less,payment_frequency_semi_monthly
0,0,1560,614,574,600,656,561,316,0,14,0.000000,36.0,3.0,3.0,2,0,0,0,0,1,0,1,0,0,1,...,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
1,0,900,708,501,550,651,563,317,0,13,0.000000,36.0,72.0,72.0,2,0,0,0,0,1,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2,620,1434,687,522,561,661,598,317,0,19,0.432357,36.0,9.5,18.0,2,0,0,0,0,1,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0
3,785,1600,616,560,552,634,591,318,0,7,0.490625,36.0,72.0,9.5,2,0,0,0,0,1,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
4,700,1360,681,603,654,659,636,319,0,10,0.514706,36.0,72.0,3.0,2,0,0,0,0,1,0,1,1,0,0,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,382,5600,492,583,634,671,627,88,4,7,0.068214,36.0,72.0,72.0,2,0,0,0,0,1,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
402,485,1000,762,638,688,714,656,88,4,8,0.485000,36.0,72.0,72.0,1,0,0,0,0,0,0,1,0,1,0,...,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0
403,873,2466,499,559,569,677,608,88,4,9,0.354015,36.0,72.0,18.0,2,0,0,0,1,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0
404,945,1300,522,572,621,620,604,88,4,11,0.726923,36.0,72.0,3.0,2,0,1,0,0,1,0,1,0,1,1,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


## Sample Run

In [66]:
# Configure constant model settings
settings = {
    'booster': 'gbtree', 
    'objective': 'binary:logistic',
    'eval_metric': 'logloss', # TODO: Suppress the warning down there. 
    'nthread': multiprocessing.cpu_count()
}

# Configure hyperparameters
# Currently building 1728 models
hypers = {
    'eta': [0.01, .10, .25, .50], # learning rate, reduces size of error adjustment
    'gamma': [0, 0.5, 5], # Raise to reduce overfitting (prevents low-magnitude leaves)    
    'min_child_weight': [1, 4, 16], # Raise to eliminate leaves with low sample weight
    'subsample': [.7, 1], # Each boosting round trains on X% of the training data
    'max_depth': [2, 4, 8], # Maximum depth of a single tree
    'reg_lambda': [0, 1], # L1 regularization
}

# Monitor hypers chosen by XGB
monitor_hypers = {
    'num_pbuffer': None,
    'num_feature': None
}

In [67]:
tree = XGBClassifier()
param = settings.update(hypers)
num_round = [2, 4, 8, 16]
tree = tree.fit(X = X_train,
                y = y_train)
tree = tree.fit(param, d_train, num_round)

# fit(X, y, *, sample_weight=None, base_margin=None, eval_set=None, eval_metric=None, early_stopping_rounds=None, verbose=True, xgb_model=None, sample_weight_eval_set=None, base_margin_eval_set=None, feature_weights=None, callbacks=None)


# # make prediction
# preds = tree.predict(d_test)

[06:24:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ValueError: y should be a 1d array, got an array of shape () instead.

## Scoring Method

OC-AUC works for a balanced dataset like this one, but isn't accurate when you expand to low default rate populations. Quote from [The Relationship between Precision-Recall and ROC Curves](https://dl.acm.org/doi/10.1145/1143844.1143874).
> The visual interpretability of ROC plots in the context of imbalanced datasets can be deceptive with respect to conclusions about the reliability of classification performance, owing to an intuitive but wrong interpretation of specificity. [Precision-recall curve] plots, on the other hand, can provide the viewer with an accurate prediction of future classification performance due to the fact that they evaluate the fraction of true positives among positive predictions

Brier Score is the best so far. Grade your probability predictions, maybe square it for the hell of it. 



## Grid Search

In [ ]:
estimator = XGBClassifier()

# apply permanent settings to the model

# initialize hyperparameters

In [ ]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'neg_brier_score',
    n_jobs = multiprocessing.cpu_count(),
    cv = 5,
    verbose=True
)

In [ ]:
grid_search.fit(X, Y)
grid_search.best_estimator_

## AUC Curve

Or do you say 'ROC Curve'?